In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from utils import get_spectrogram

In [2]:
tf_records = ["data/raw/" + filename for filename in os.listdir("./data/raw")]
tf_records

['data/raw/nsynth-test.tfrecord-00000-of-00008',
 'data/raw/nsynth-test.tfrecord-00001-of-00008',
 'data/raw/nsynth-test.tfrecord-00002-of-00008',
 'data/raw/nsynth-test.tfrecord-00003-of-00008',
 'data/raw/nsynth-test.tfrecord-00004-of-00008',
 'data/raw/nsynth-test.tfrecord-00005-of-00008',
 'data/raw/nsynth-test.tfrecord-00006-of-00008',
 'data/raw/nsynth-test.tfrecord-00007-of-00008']

In [3]:
raw_dataset = tf.data.TFRecordDataset(tf_records)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [4]:
info = pd.DataFrame(columns=["instrument_family", "instrument_source", "instrument", "pitch", "note"])
audio = []
for raw_record in raw_dataset.take(-1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  row = [ 
    example.features.feature["instrument_family"].int64_list.value[0],
    example.features.feature["instrument_source"].int64_list.value[0],
    example.features.feature["instrument"].int64_list.value[0],
    example.features.feature["pitch"].int64_list.value[0],
    example.features.feature["note"].int64_list.value[0] 
  ]
  info.loc[len(info)] = row
  audio.append(example.features.feature["audio"].float_list.value)

In [5]:
info_train = info[:3072]
info_test = info[3072:]
audio_train = audio[:3072]
audio_test = audio[3072:]

In [8]:
spectrograms_train = [get_spectrogram(waveform) for waveform in audio_train]
spectrograms_test = [get_spectrogram(waveform) for waveform in audio_test]

In [6]:
labels_src_train = info_train["instrument_source"].to_list()
labels_src_test = info_test["instrument_source"].to_list()
labels_instr_train = info_train["instrument_family"].to_list()
labels_instr_test = info_test["instrument_family"].to_list()

In [10]:
spectrograms_train = tf.convert_to_tensor(spectrograms_train)
spectrograms_test = tf.convert_to_tensor(spectrograms_test)

In [ ]:
labels_src_train = tf.convert_to_tensor(labels_src_train)
labels_src_test = tf.convert_to_tensor(labels_src_test)

In [7]:
labels_instr_train = tf.convert_to_tensor(labels_instr_train)
labels_instr_test = tf.convert_to_tensor(labels_instr_test)

In [7]:
audio_train = tf.convert_to_tensor(audio_train)
audio_test = tf.convert_to_tensor(audio_test)

In [11]:
spectrograms_train = tf.Variable(spectrograms_train)
spectrograms_test = tf.Variable(spectrograms_test)

In [ ]:
labels_src_train = tf.Variable(labels_src_train)
labels_src_test = tf.Variable(labels_src_test)

In [8]:
labels_instr_train = tf.Variable(labels_instr_train)
labels_instr_test = tf.Variable(labels_instr_test)

In [9]:
audio_train = tf.Variable(audio_train)
audio_test = tf.Variable(audio_test)

In [15]:
tf.saved_model.save(spectrograms_train, "data/processed/spectrograms_train")
tf.saved_model.save(spectrograms_test, "data/processed/spectrograms_test")


INFO:tensorflow:Assets written to: data/processed/labels_train\assets
INFO:tensorflow:Assets written to: data/processed/labels_test\assets


In [ ]:
tf.saved_model.save(labels_src_train, "data/processed/labels_src_train")
tf.saved_model.save(labels_src_test, "data/processed/labels_src_test")

In [9]:
tf.saved_model.save(labels_instr_train, "data/processed/labels_instr_train")
tf.saved_model.save(labels_instr_test, "data/processed/labels_instr_test")

INFO:tensorflow:Assets written to: data/processed/labels_instr_train\assets
INFO:tensorflow:Assets written to: data/processed/labels_instr_test\assets


In [10]:
tf.saved_model.save(audio_train, "data/processed/audio_train")
tf.saved_model.save(audio_test, "data/processed/audio_test")

INFO:tensorflow:Assets written to: data/processed/audio_train\assets
INFO:tensorflow:Assets written to: data/processed/audio_test\assets
